# 4d. Get osm, project sidewalk and traffic sign crossings

In [ ]:
# Standard library and path imports
import set_path

# Third-party library imports
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import momepy
import networkx as nx
from branca.element import Template, MacroElement
import folium

# Local or project-specific imports
import plot_utils
import crossing_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Import walking network 

In [ ]:
# Get basic pedestrian network with widths
gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=st.CRS)
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=st.CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes.reset_index()
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')

In [ ]:
# Calculate connected components for pedestrian network
cc_network = list(nx.connected_components(graph_network))
nodes_in_edge = [line.coords[0] for line in gdf_network['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_network)) if node in cc_network[i]]
gdf_network_nodes['cc'] = [x for y in zip(cc_ids, cc_ids) for x in y]

## Create crossings with OSM crossing features

### Import osm crossing features

In [ ]:
# Import osm crossing features
gdf_osm_cf = gpd.read_file(cf.output_osm_crossing_features).to_crs(crs=st.CRS)

### Select crossing features and nodes to keep 


In [ ]:
# Create network and crossing feature dataframes
gdf_osm_cf_sel = gdf_osm_cf.loc[gdf_osm_cf['highway'] != 'cycleway']
gdf_osm_nodes = gdf_osm_cf_sel[gdf_osm_cf_sel['element_type'] == 'node']
gdf_osm_edges = gdf_osm_cf_sel[gdf_osm_cf_sel['element_type'] == 'way']
gdf_osm_edges = gdf_osm_edges.drop(columns=gdf_osm_edges.columns.difference(['geometry']))

# Remove crossing nodes that already are on a crossing edge
if not gdf_osm_edges.empty:
    for i, row in gdf_osm_nodes.iterrows():
        gdf_osm_nodes.loc[i, 'min_dis'] = gdf_osm_edges.distance(row['geometry']).min()
    gdf_osm_nodes = gdf_osm_nodes.loc[gdf_osm_nodes['min_dis'] <= 0.01]

### Add osm crosswalk edges to the network

In [ ]:
# # Current heuristic: find closest node in the network and connect outer points of crosswalk edge to that point

# Get outer nodes of crossing edges and connect to network
gdf_osm_edge_nodes = gpd.GeoDataFrame(geometry=gdf_osm_edges['geometry'].boundary.explode(index_parts=True), crs=st.CRS).reset_index()
gdf_osm_edge_nodes['x'], gdf_osm_edge_nodes['y'] = gdf_osm_edge_nodes.geometry.x, gdf_osm_edge_nodes.geometry.y
gdf_new_edges_from_osm_edges = crossing_utils.get_connections(gdf_osm_edge_nodes, gdf_network_nodes, max_dist=10, max_connections=2, 
                                                                crs=st.CRS, include_cc_rule=True, cc_column='cc')
print('Number of new edges from crosswalk edge to network node:', len(gdf_new_edges_from_osm_edges))

### Add osm crosswalk nodes to the network

In [ ]:
# Get coordinates of crossing feature nodes
gdf_osm_nodes['x'], gdf_osm_nodes['y'] = gdf_osm_nodes.geometry.x, gdf_osm_nodes.geometry.y

# Connect osm crosswalk nodes to eachother
gdf_self_edges = crossing_utils.get_connections(gdf_osm_nodes, gdf_osm_nodes, max_dist=10, max_connections=2, crs=st.CRS)
print('Number of new edges from crossing node to crossing node:', len(gdf_self_edges))

# Connect osm crosswalk nodes to network
gdf_network_edges = crossing_utils.get_connections(gdf_osm_nodes, gdf_network_nodes, max_dist=10, max_connections=3, crs=st.CRS, include_cc_rule=True, cc_column='cc')
print('Number of new edges from osm node to network node:', len(gdf_network_edges))

### visualize OSM crossings

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_osm_edges, style_function=lambda x: {"color": "orange", "weight": 4}).add_to(map)
geo_j = folium.GeoJson(gdf_new_edges_from_osm_edges, style_function=lambda x: {"color": "red", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_osm_cf_sel.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_osm_cf_sel, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Plot map
template = plot_utils.gen_template()
macro = MacroElement()
macro._template = Template(template)
map.get_root().add_child(macro)

map

### Store OSM crossings

In [ ]:
gdf_edges = pd.concat([gdf_osm_edges, gdf_new_edges_from_osm_edges, gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_osm_crossings, driver='GPKG')

## Create crossings with Project Sidewalk crossing features

### Import Project sidewalk crossing features

In [ ]:
# Import project sidewalk crossing features
gdf_ps_cf = gpd.read_file(cf.output_project_sidewalk_crossing_features)

### Select crossing features and nodes to keep

In [ ]:
# Create network and crossing feature dataframes
gdf_ps_nodes = gdf_ps_cf.loc[gdf_ps_cf['label_type'] == 'Crosswalk']

# TODO Check if ['CurbRamp' 'Obstacle' 'Signal' 'NoCurbRamp' 'SurfaceProblem'] nodes are useful to find crossings as well

### Add project sidewalk nodes to network

In [ ]:
# Get coordinates of crossing feature and network nodes
gdf_ps_nodes['x'], gdf_ps_nodes['y'] = gdf_ps_nodes.geometry.x, gdf_ps_nodes.geometry.y

# Connect project sidewalk crosswalk nodes to eachother
gdf_self_edges = crossing_utils.get_connections(gdf_ps_nodes, gdf_ps_nodes, max_dist=10, max_connections=2, crs=st.CRS)
print('Number of new edges from crosswalk node to crosswalk node:', len(gdf_self_edges))

# Connect project sidewalk crosswalk nodes to network
gdf_network_edges = crossing_utils.get_connections(gdf_ps_nodes, gdf_network_nodes, max_dist=10, max_connections=3, crs=st.CRS, include_cc_rule=True, cc_column='cc')
print('Number of new edges from crosswalk node to network node:', len(gdf_network_edges))

### Visualize Project Sidewalk crossings

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network edges and crossing nodes
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_ps_nodes.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_ps_nodes, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Save map
map.get_root().add_child(macro)
map

### Store Project Sidewalk crossings

In [ ]:
gdf_edges = pd.concat([gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_project_sidewalk_crossings, driver='GPKG')

## Add crossings with traffic sign features 

### Import traffic sign features

In [ ]:
# Import project traffic sign features
gdf_ts_nodes = gpd.read_file(cf.output_traffic_sign_crossing_features)

### Add traffic sign nodes to network

In [ ]:
# Get coordinates of crossing feature and network nodes
gdf_ts_nodes['x'], gdf_ts_nodes['y'] = gdf_ts_nodes.geometry.x, gdf_ts_nodes.geometry.y

# Connect traffic sign crosswalk nodes to eachother
gdf_self_edges = crossing_utils.get_connections(gdf_ts_nodes, gdf_ts_nodes, max_dist=20, max_connections=2, crs=st.CRS)
print('Number of new edges from traffic sign node to traffic sign node:', len(gdf_self_edges))

# Connect traffic sign crosswalk nodes to network
gdf_network_edges = crossing_utils.get_connections(gdf_ts_nodes, gdf_network_nodes, max_dist=12, max_connections=3, crs=st.CRS, include_cc_rule=True, cc_column='cc')
print('Number of new edges from traffic sign node to network node:', len(gdf_network_edges))

### Visualize traffic sign crossings

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network edges and crossing nodes
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_network_edges, style_function=lambda x: {"color": "yellow", "weight": 6}).add_to(map)
geo_j = folium.GeoJson(gdf_self_edges, style_function=lambda x: {"color": "lightgreen", "weight": 6}).add_to(map)

# Add crossing edges and nodes
feature_names = gdf_ts_nodes.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(gdf_ts_nodes, style_function=lambda x: {"color": "orange", "weight": 4}, tooltip=tooltip).add_to(map)

# Save map
map.get_root().add_child(macro)
map

### Store traffic sign crossings

In [ ]:
gdf_edges = pd.concat([gdf_self_edges, gdf_network_edges], ignore_index=True, sort=False)
gdf_edges.to_file(cf.output_traffic_sign_crossings, driver='GPKG')